In [7]:
import os
import win32com.client as com
import pandas as pd

In [8]:
# 1. Visum 파일 경로 설정
base_path = r"C:/Digital Twin Simulation network/VISUM/"
version_file = "강릉시 전국 전일 최종본.ver"
version_path = os.path.join(base_path, version_file)

if not os.path.isfile(version_path):
    raise FileNotFoundError(f"Visum 파일을 찾을 수 없습니다: {version_path}")

In [9]:
# 2. VISUM 객체 생성 및 버전 로드
Visum = com.Dispatch("Visum.Visum.22")
Visum.LoadVersion(version_path)
print(f"✅ VISUM 버전 파일 로드 완료: {version_path}")

✅ VISUM 버전 파일 로드 완료: C:/Digital Twin Simulation network/VISUM/강릉시 전국 전일 최종본.ver


In [10]:
# 3. Links 속성 읽기
attrs = [
    "AREA",                # 교동1, 송정2, 도심3, 경포4
    "SUBAREA",             # subarea
    "LINK_ID",             # link_id 표준링크 아이디 '2520022301, 2520679101' or '2520019507'
    "ROAD_NAME",           # road_name
    "UPDOWN",              # updown
    "VolCapRatioPrT(AP)",  # vc
    "VolVehPrT(AP)",       # volume
    "TCur_PrTSys(a)",      # speed
]

rows = Visum.Net.Links.GetMultipleAttributes(attrs, True)

In [13]:
# 4. DataFrame으로 변환 (LINK_ID가 없는 것은 제외 + 예외 ID 체크)
invalid_ids = set()
records = []

for row in rows:
    rec = dict(zip(attrs, row))
    raw_id = rec.get("LINK_ID")

    if not raw_id:  # None, 빈 문자열 등 제외
        continue

    # 쉼표로 분리된 경우 처리
    link_ids = [x.strip() for x in str(raw_id).split(",") if x.strip()]

    # 모든 link_id가 10자리 숫자인 경우만 정상으로 인정
    if all(len(x) == 10 and x.isdigit() for x in link_ids):
        records.append(rec)
    else:
        invalid_ids.add(raw_id)  # 예외값 저장

# DataFrame 생성
df = pd.DataFrame.from_records(records)

# 5. 상위 10개 출력
print("✅ 링크 속성 데이터 (상위 10개):")
display(df.head(50))

# 6. 행/열 개수 출력
print(f"📊 DataFrame 크기: {len(df)} 행 × {len(df.columns)} 열")

# 7. 예외 link_id 출력
if invalid_ids:
    print("⚠️ 예외 LINK_ID 목록:")
    for bad in invalid_ids:
        print("   ", bad)
else:
    print("✅ 모든 LINK_ID가 10자리 숫자 조건을 만족함")

✅ 링크 속성 데이터 (상위 10개):


,AREA,SUBAREA,LINK_ID,ROAD_NAME,UPDOWN,VolCapRatioPrT(AP),VolVehPrT(AP),TCur_PrTSys(a)
0,3.0,,2520174700,경강로,0.0,0.0,0.0,2.040720e+01
1,3.0,,2520143400,경강로1804번길,1.0,0.0,0.0,2.148425e+01
2,3.0,,2520019610,경강로,0.0,0.0,0.0,2.932654e+00
3,3.0,,2520019510,경강로,1.0,0.0,0.0,2.932654e+00
4,3.0,,2520174901,공제로,1.0,0.0,0.0,4.306960e+01
5,3.0,,2520175002,공제로,0.0,0.0,0.0,4.306960e+01
6,3.0,,2520019608,경강로,0.0,0.0,0.0,1.179589e+01
7,3.0,,2520019508,경강로,1.0,0.0,0.0,1.179589e+01
8,3.0,,2520019508,경강로,1.0,0.0,0.0,1.132071e+01
9,3.0,,2520019608,경강로,0.0,0.0,0.0,1.132071e+01


📊 DataFrame 크기: 1945 행 × 8 열
✅ 모든 LINK_ID가 10자리 숫자 조건을 만족함
